In [2]:
%matplotlib inline
import collections
import h5py
import matplotlib.pyplot as plt
import numpy as np
import numba
from numba.typed import Dict, List
import pickle

dendrites = h5py.File('../cache/dendrites.hdf5', 'r')
axons = h5py.File('../cache/axons.hdf5', 'r')

In [4]:
def get_dendrite_map(dendrites):
    # Build a map from agglomerate ID to neuron id
    # First for dendrites
    agg_ids = list(dendrites['dendrites']['agglomerate'])
    neuron_ids = list(dendrites['dendrites']['neuronId'])
    agg_to_neuron_id = {k: neuron_ids[int(k) - 1] for k in agg_ids}

    neuron_segments = collections.defaultdict(lambda: [])
    neuron_aggs = collections.defaultdict(lambda: [])

    for agg in list(dendrites['dendrites']['agglomerate'].keys()):
        if agg in agg_to_neuron_id and agg_to_neuron_id[agg] > 0:
            neuron_id = agg_to_neuron_id[agg]
            neuron_segments[neuron_id] += list(dendrites['dendrites']['agglomerate'][agg])
            neuron_aggs[neuron_id].append(agg)

    # Add the axons, if available.
    dendrites_classes = np.array(dendrites['dendrites']['class'])

    neuron_map = {}
    for neuron_id, segment_ids in neuron_segments.items():
        for segment_id in segment_ids:
            neuron_map[segment_id] = neuron_id
    return neuron_segments, neuron_map, neuron_aggs

dendrite_segments, dendrite_map, dendrite_aggs = get_dendrite_map(dendrites)

dendrite_data = {'dendrite_segments': dict(dendrite_segments), 
                 'dendrite_map': dendrite_map,
                 'dendrite_aggs': dict(dendrite_aggs)}
with open('dendrite_to_neuron.pkl', 'wb') as f:
    pickle.dump(dendrite_data, f)

In [9]:
def get_neuron_map_with_axons(dendrites, axons):
    neurons = 0
    neurons_with_ais = 0
    neurons_with_axons = 0
    
    agg_ids = list(dendrites['dendrites']['agglomerate'])
    neuron_ids = list(dendrites['dendrites']['neuronId'])
    agg_to_neuron_id = {k: neuron_ids[int(k) - 1] for k in agg_ids}

    neuron_segments = collections.defaultdict(lambda: [])
    neuron_aggs = collections.defaultdict(lambda: [])
    dendrite_classes = np.array(dendrites['dendrites']['class'])
    axon_classes = np.array(axons['axons']['class'])
    
    unique_neuron_ids = np.unique(neuron_ids)
    assert unique_neuron_ids[0] == 0
    unique_neuron_ids = unique_neuron_ids[1:]
    
    for neuron_id in unique_neuron_ids:
        neurons += 1
        
        # Find the AIS for this "dendrite"
        ais_agg = [k for k, v in agg_to_neuron_id.items() if v == neuron_id and dendrite_classes[int(k) - 1] == 2]

        if ais_agg:
            assert len(ais_agg) == 1
            neurons_with_ais += 1
            ais_segments = np.array(dendrites['dendrites']['skeleton'][ais_agg[0]]['segIds'])
            ais_segments = np.unique(ais_segments[ais_segments > 0])

            # Find an axon whose segments overlap this AIS.
            found_agg = False
            for i, agglomerate in enumerate(axons['axons']['skeleton'].keys()):
                probe = np.unique(np.array(axons['axons']['skeleton'][agglomerate]['segIds']))
                if len(ais_segments) + len(probe) > len(np.unique(np.concatenate((ais_segments, probe)))):
                    print("Found potential axon")
                    axon_class = axon_classes[int(agglomerate) - 1]
                    print(axon_class)
                    if axon_class in (1, 3, 4):
                        print("Good axon")
                    else:
                        continue
                    # Check if there is no synapse between the two elements.
                    print(len(list(axons['axons']['agglomerate'][agglomerate])))
                    neurons_with_axons += 1
                    found_agg = True
                    neuron_segments[neuron_id] += list(axons['axons']['agglomerate'][agglomerate])
                    neuron_aggs[neuron_id].append(agglomerate)
                    
    neuron_map_with_axons = {}
    for neuron_id, segment_ids in neuron_segments.items():
        for segment_id in segment_ids:
            neuron_map_with_axons[segment_id] = neuron_id
    return neuron_segments, neuron_map_with_axons, neuron_aggs

axon_segments, axon_map, axon_aggs = get_neuron_map_with_axons(dendrites, axons)

axon_data = {'axon_segments': dict(axon_segments), 
             'axon_map': axon_map,
             'axon_aggs': dict(axon_aggs)}

with open('axon_to_neuron.pkl', 'wb') as f:
    pickle.dump(axon_data, f)

Found potential axon
1
Good axon
367
Found potential axon
4
Good axon
1
Found potential axon
4
Good axon
57
Found potential axon
4
Good axon
547
Found potential axon
4
Good axon
36
Found potential axon
4
Good axon
74
Found potential axon
4
Good axon
27
Found potential axon
4
Good axon
61
Found potential axon
4
Good axon
72
Found potential axon
4
Good axon
153
Found potential axon
1
Good axon
696
Found potential axon
1
Good axon
1727
Found potential axon
1
Good axon
797
Found potential axon
4
Good axon
153
Found potential axon
1
Good axon
325
Found potential axon
4
Good axon
40
Found potential axon
4
Good axon
250
Found potential axon
4
Good axon
84
Found potential axon
4
Good axon
25
Found potential axon
4
Good axon
426
Found potential axon
4
Good axon
7
Found potential axon
1
Good axon
3141
Found potential axon
4
Good axon
328
Found potential axon
4
Good axon
8
Found potential axon
4
Good axon
20
Found potential axon
4
Good axon
2
Found potential axon
4
Good axon
69
Found potential ax

In [10]:
axon_aggs

defaultdict(<function __main__.get_neuron_map_with_axons.<locals>.<lambda>()>,
            {1: ['4685'],
             3: ['25677'],
             4: ['16082'],
             7: ['33689'],
             8: ['30215'],
             10: ['2538'],
             13: ['22792'],
             14: ['38957'],
             19: ['16757'],
             20: ['16980'],
             21: ['33691'],
             22: ['114'],
             25: ['982'],
             27: ['12579'],
             29: ['12392', '12393'],
             30: ['21495'],
             31: ['13735'],
             32: ['18132', '8279'],
             38: ['26115'],
             39: ['33693'],
             40: ['7889'],
             42: ['25054', '29321', '30415'],
             43: ['24103'],
             49: ['1856', '2616'],
             51: ['33692'],
             52: ['23347'],
             53: ['5458'],
             54: ['29670'],
             55: ['13802'],
             60: ['3866'],
             63: ['11269'],
             64: ['24400'